In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.utils.vis_utils import plot_model
import sklearn
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import math
import matplotlib.pyplot as plt

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

# Set seeds to make the experiment more reproducible.
#from tensorflow import set_random_seed
from numpy.random import seed
#set_random_seed(1)
seed(1)

train = pd.read_csv('TestData/full_trainset.csv',',')

train.describe()

,Assembly,Material,OpenOrders,NewOrders,TotalWork,TotalSetup,SumDuration,SumOperations,ProductionOrders,CycleTime
count,176314.000000,1.763140e+05,176314.000000,176314.000000,176314.000000,176314.000000,176314.000000,176314.000000,176314.000000,176314.000000
mean,83.748200,7.055790e+08,14.283738,2.094871,4.910643,9.555901,221.342780,21.930902,4.386180,2933.339803
std,21.765351,1.831323e+08,9.217014,1.595109,2.053752,2.552856,87.776384,8.740822,1.748164,1380.312452
min,5.022024,6.544642e+07,2.000000,0.000000,0.000000,0.000000,145.000000,15.000000,3.000000,1457.000000
25%,72.765690,6.127527e+08,10.000000,1.000000,3.570000,8.480000,153.000000,15.000000,3.000000,2001.000000
50%,92.376175,7.767417e+08,12.000000,2.000000,4.790000,10.280000,197.000000,20.000000,4.000000,2450.000000
75%,99.854836,8.421148e+08,15.000000,3.000000,6.060000,11.400000,255.000000,25.000000,5.000000,3439.000000
max,109.961290,9.245294e+08,91.000000,12.000000,11.480000,13.980000,442.000000,45.000000,9.000000,14855.000000


In [2]:
train

,Assembly,Material,OpenOrders,NewOrders,TotalWork,TotalSetup,SumDuration,SumOperations,ProductionOrders,CycleTime
0,18.279465,129014860,10,1,2.24,5.10,147,15,3,1661
1,18.279465,129014860,10,1,2.24,5.10,442,45,9,6461
2,18.279465,129014860,10,1,2.24,5.10,153,15,3,1983
3,19.841406,187106750,11,3,3.90,9.03,158,15,3,2300
4,19.841406,187106750,11,3,3.90,9.03,254,25,5,3478
...,...,...,...,...,...,...,...,...,...,...
176309,106.379940,888429800,11,3,6.17,11.66,152,15,3,1935
176310,106.379940,888429800,11,3,6.17,11.66,147,15,3,1671
176311,106.566734,889996700,13,2,5.20,10.67,156,15,3,1636
176312,106.628880,886085570,12,1,5.57,10.58,158,15,3,2247


In [3]:
split_fraction = 0.70
train_split = int(split_fraction * int(train.shape[0]))
step = 1

#past = 0
#future = 8
learning_rate = 0.0005
batch = 32
epochs = 100
mean = 0
std = 0

def normalize(data, train_split):
    global mean
    global std
    data_mean = data[:train_split].mean(axis=0)
    mean = data_mean
    data_std = data[:train_split].std(axis=0)
    std = data_std
    return (data - data_mean) / data_std

In [4]:
titles = ['Assembly',
          'Material',
#           'OpenOrders',
#           'NewOrders',
          'TotalWork',
          'TotalSetup',
          'SumDuration',
          'SumOperations',
          'ProductionOrders',
          'CycleTime']
#  
# for c in train.columns:
#     titles.append(c);
    
features = train[titles]
features = normalize(features.values, train_split)
features = pd.DataFrame(features)
features.head()

,0,1,2,3,4,5,6,7
0,-3.028536,-3.168889,-1.310899,-1.798170,-0.846272,-0.792129,-0.792129,-0.917680
1,-3.028536,-3.168889,-1.310899,-1.798170,2.516269,2.640889,2.640889,2.360140
2,-3.028536,-3.168889,-1.310899,-1.798170,-0.777881,-0.792129,-0.792129,-0.697793
3,-2.956266,-2.850691,-0.547436,-0.213985,-0.720889,-0.792129,-0.792129,-0.481320
4,-2.956266,-2.850691,-0.547436,-0.213985,0.373361,0.352210,0.352210,0.323112


In [5]:
train_data = features.loc[0 : train_split - 1] #Training Data
val_data = features.loc[train_split:] #Validation Data
train_data

,0,1,2,3,4,5,6,7
0,-3.028536,-3.168889,-1.310899,-1.798170,-0.846272,-0.792129,-0.792129,-0.917680
1,-3.028536,-3.168889,-1.310899,-1.798170,2.516269,2.640889,2.640889,2.360140
2,-3.028536,-3.168889,-1.310899,-1.798170,-0.777881,-0.792129,-0.792129,-0.697793
3,-2.956266,-2.850691,-0.547436,-0.213985,-0.720889,-0.792129,-0.792129,-0.481320
4,-2.956266,-2.850691,-0.547436,-0.213985,0.373361,0.352210,0.352210,0.323112
...,...,...,...,...,...,...,...,...
123414,-1.913051,-1.959928,-0.414060,-0.222047,0.373361,0.352210,0.352210,-0.230020
123415,-1.913051,-1.959928,-0.414060,-0.222047,-0.686694,-0.792129,-0.792129,-0.593312
123416,-1.913051,-1.959928,-0.414060,-0.222047,-0.720889,-0.792129,-0.792129,-0.919045
123417,-1.922156,-1.984341,0.206829,0.640588,-0.686694,-0.792129,-0.792129,-0.864415


In [6]:
#start = past + future
start = 0
end = start + train_split

x_train = train_data[[i for i in range(len(titles) - 1)]].values
y_train = features.iloc[start:end][[len(titles)-1]]

#sequence_length = int(past / step)
sequence_length = 1
x_train[0]

array([-3.02853594, -3.16888922, -1.31089927, -1.79816985, -0.84627184,
       -0.79212888, -0.79212888])

In [7]:
dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train,
    y_train,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch
)

In [8]:
label_start = train_split
valRange = int(train.shape[0]) - train_split

# x_val = val_data.iloc[[i for i in range(valRange)]].values
x_val = val_data[[i for i in range(len(titles) - 1)]].values
# x_val = val_data.iloc[[i for i in range(49)]].values
y_val = features.iloc[label_start:][[len(titles)-1]]

dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch
)

for batch in dataset_train.take(1):
    inputs, targets = batch
print(x_val)
print(y_val)
print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)
print(inputs)

[[-1.92215582 -1.98434119  0.20682923 ...  2.23130759  2.06871899
   2.06871899]
 [-1.92215582 -1.98434119  0.20682923 ...  1.01167426  0.92437984
   0.92437984]
 [-1.92215582 -1.98434119  0.20682923 ...  0.36196305  0.35221027
   0.35221027]
 ...
 [ 1.0564665   0.99938528  0.05045721 ... -0.74368586 -0.79212888
  -0.79212888]
 [ 1.05934196  0.97796208  0.22062677 ... -0.72088898 -0.79212888
  -0.79212888]
 [ 1.05967325  0.97545848 -0.97515933 ... -0.7550843  -0.79212888
  -0.79212888]]
               7
123419  1.180808
123420  0.194047
123421  0.362036
123422 -1.005771
123423 -0.867147
...          ...
176309 -0.730571
176310 -0.910851
176311 -0.934752
176312 -0.517512
176313 -0.906754

[52895 rows x 1 columns]
Input shape: (32, 1, 7)
Target shape: (32, 1)
tf.Tensor(
[[[-3.02853594 -3.16888922 -1.31089927 -1.79816985 -0.84627184
   -0.79212888 -0.79212888]]

 [[-3.02853594 -3.16888922 -1.31089927 -1.79816985  2.51626864
    2.64088856  2.64088856]]

 [[-3.02853594 -3.16888922 -1.31089

In [9]:
# Long Short Term Memory - Model als Methodik mit Adam --> stochastic gradient descent algorithm

inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
dropout1 = keras.layers.Dropout(0.4)(inputs)
hidden1 = keras.layers.LSTM(10)(dropout1)
dropout2 = keras.layers.Dropout(0.4)(hidden1)
outputs = keras.layers.Dense(1, activation='tanh')(dropout2)
learning_rate = 0.001
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.99), loss=keras.losses.MeanSquaredError(), metrics=keras.metrics.MeanAbsoluteError())
model.summary()
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
model.save("kerasModel")

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 7)]            0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 7)              0         
_________________________________________________________________
lstm (LSTM)                  (None, 10)                720       
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 11        
Total params: 731
Trainable params: 731
Non-trainable params: 0
_________________________________________________________________
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ',

INFO:tensorflow:Assets written to: kerasModel\assets


INFO:tensorflow:Assets written to: kerasModel\assets


In [10]:
path_checkpoint = "kerasModel/simpleModelCheckpoint.h5"

def scheduler(epoch, lr):
    return 0.0001
    if lr > 0.004:
        return lr - 0.0002
    else:
        if lr > 0.0004:
            return lr - 0.000001
        else:            
            return 0.0001

es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=10)

lr_scheduler = keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_best_only=True
)

epochs = 4000

history = model.fit(
    dataset_train,
    epochs=epochs,
    validation_data=dataset_val,
    callbacks=[es_callback,modelckpt_callback, lr_scheduler]
)

Epoch 1/4000

Epoch 00001: LearningRateScheduler reducing learning rate to 0.0001.
3857/3857 [==============================] - 10s 2ms/step - loss: 0.8718 - mean_absolute_error: 0.6523 - val_loss: 0.1957 - val_mean_absolute_error: 0.3393

Epoch 00001: val_loss improved from inf to 0.19567, saving model to kerasModel\simpleModelCheckpoint.h5
Epoch 2/4000

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0001.
3856/3857 [============================>.] - ETA: 0s - loss: 0.7241 - mean_absolute_error: 0.5715

KeyboardInterrupt: 

In [ ]:
def visualize_loss(history, title):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(len(loss))
    plt.figure()
    plt.plot(epochs, loss, "b", label="Training loss")
    plt.plot(epochs, val_loss, "r", label="Validation loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()
    
visualize_loss(history, "Training and Validation loss")

In [ ]:
def show_plot(plot_data, delta, title):
    labels = ["History", "True Future", "Model Prediction"]
    marker = [".-", "rx", "go"]
    time_steps = list(range(-(plot_data[0].shape[0]), 0))
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, val in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future + 5) * 2])
    plt.xlabel("Time-Step")
    plt.show()
    return

for x, y in dataset_val.take(1):
    show_plot(
        [x[0][:, 0].numpy(), y[0].numpy(), model.predict(x)[0]],
        1,
        "Single Step Prediction",
    )

In [ ]:
def denormalize(value):
    data_mean = mean[len(titles)-1]
    data_std = std[len(titles)-1]
    return value*data_std+data_mean

for x, y in dataset_val.take(10):
    predictionData = model.predict(x)
    denormalized_predictionData = denormalize(predictionData)[0]
    actualValue = y[0].numpy()
    print("predicted denormalized:", denormalized_predictionData)
    print("   actual denormalized:", denormalize(actualValue))
    print("             predicted:", predictionData[0])
    print("                actual:", actualValue)

# for x, y in datasetPredTest.take(1):
#     show_plot(
#         [x[0][:, 0].numpy(), y[0].numpy(), model.predict(x)[0]],
#         1,
#         "Single Step Prediction",
#     )

In [ ]:
def normalize(data):
    return (data - mean) / std

predTest = pd.read_csv('TestData/testData1.csv',';')
mean = train[titles].values.mean(axis=0)
std = train[titles].values.std(axis=0)
featuresPredTest = predTest[titles]
predTestRange = int(featuresPredTest.shape[0])
featuresPredTest = normalize(featuresPredTest.values)
featuresPredTest = pd.DataFrame(featuresPredTest)
x_predTest = featuresPredTest[[i for i in range(7)]].values
y_predTest = featuresPredTest.iloc[0:][[7]]

datasetPredTest = keras.preprocessing.timeseries_dataset_from_array(
    x_predTest,
    y_predTest,
    sequence_length=1,
    sampling_rate=1,
    batch_size=1
)

for x, y in datasetPredTest.take(10):
    predictionData = model.predict(x)
    denormalized_predictionData = denormalize(predictionData)
    print("predicted denormalized:", denormalized_predictionData)
    print("             predicted:", predictionData)
    print("                actual:", y[0].numpy())

In [ ]:
# for x,y in datasetPredTest.take(2):
#     print(x[0][0].numpy(
normalize(featuresPredTest.values)

In [ ]:
print(mean[9], std[9])

In [ ]:
x_train

In [ ]:
x_train.shape

In [ ]:
y_train

In [ ]:
y_train.shape

In [ ]:
mean

In [ ]:
std

In [ ]:
train[titles].values.mean(axis=0)

In [ ]:
test = [[[3.96250010e+00, 5.92916600e+07, 7.00000000e+00, 7.00000000e+00, 4.53000021e+00, 1.08999996e+01, 1.99000000e+02, 2.00000000e+01, 4.00000000e+00, 0.00000000e+00]]]

In [ ]:
test

In [ ]:
(test-train[titles].values.mean(axis=0))/train[titles].values.std(axis=0)

In [ ]:
train[titles].values.mean(axis=0)

In [ ]:
train[titles].values.std(axis=0)

In [ ]:
targets

In [ ]:
train_data.shape

In [14]:
y_val.values

array([[ 1.18080767],
       [ 0.19404747],
       [ 0.36203571],
       ...,
       [-0.93475157],
       [-0.51751248],
       [-0.90675353]])

In [13]:
x_val.shape

(52895, 7)